<a href="https://colab.research.google.com/github/bojanabojchovska/CommentsSentimentAnalysis/blob/main/CommentsSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [ ]:
pip install gdown==v4.6.3

In [ ]:
!gdown 1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF


Downloading...
From: https://drive.google.com/uc?id=1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF
To: /content/IMDB_5.csv
100% 323k/323k [00:00<00:00, 4.04MB/s]


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/IMDB_5.csv')

In [ ]:
data

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",negative
1,Why do movie makers always go against the auth...,negative
2,No puedo creer que aquellos alabando esta pelí...,negative
3,Esta película realmente usó sus ubicaciones bi...,positive
4,"De manera extraña, esta película nunca llegó a...",positive
...,...,...
245,Obabakoak es un montón de cuentos con un único...,negative
246,Todo el mundo civilizado por ahora sabe que es...,positive
247,"Soy un enorme fan de las películas de guerra, ...",negative
248,"De acuerdo, aquí está el trato: si te ofenden ...",positive


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
data['sentiment'] =  encoder.fit_transform(data['sentiment'])

In [ ]:
data.sentiment.value_counts()

1    132
0    118
Name: sentiment, dtype: int64

In [ ]:
data

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",0
1,Why do movie makers always go against the auth...,0
2,No puedo creer que aquellos alabando esta pelí...,0
3,Esta película realmente usó sus ubicaciones bi...,1
4,"De manera extraña, esta película nunca llegó a...",1
...,...,...
245,Obabakoak es un montón de cuentos con un único...,0
246,Todo el mundo civilizado por ahora sabe que es...,1
247,"Soy un enorme fan de las películas de guerra, ...",0
248,"De acuerdo, aquí está el trato: si te ofenden ...",1


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(data['review_es'],data['sentiment'],test_size=0.2)

In [ ]:
df_train = pd.concat([X_train,Y_train],axis=1)
df_test= pd.concat([X_test,Y_test],axis=1)

In [ ]:
df_train.columns = ['review_es','label']
df_test.columns = ['review_es','label']

In [ ]:
df_train.to_csv('train.csv',index=False)
df_test.to_csv('test.csv',index=False)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})


In [ ]:
encoded_dataset = dataset.map(lambda t: tokenizer(t['review_es'],  truncation=True), batched=True,load_from_cache_file=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased',num_labels =2)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'bert.pooler.dense.weight', 'classifier.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

encoded_dataset


DatasetDict({
    train: Dataset({
        features: ['review_es', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review_es', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
})

In [ ]:
!pip install transformers[torch]

In [ ]:
! pip install accelerate -U


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
arg = TrainingArguments(
    "sentiment",
    num_train_epochs=2,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=42
)

In [ ]:
trainer = Trainer(
    model=model,
    args=arg,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=50, training_loss=0.3709107971191406, metrics={'train_runtime': 1567.5432, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.032, 'total_flos': 97145534971200.0, 'train_loss': 0.3709107971191406, 'epoch': 2.0})

In [ ]:
import numpy as np


In [ ]:
y_pred = trainer.predict(encoded_dataset['test'])

In [ ]:
y_pred = y_pred.predictions

In [ ]:

y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]

In [ ]:
from sklearn.metrics import classification_report,f1_score

In [ ]:
f1_score(df_test['label'],y_pred,average='macro')


0.7996794871794872

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(predictions.label_ids, preds))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62        20
           1       1.00      0.20      0.33        30

    accuracy                           0.52        50
   macro avg       0.73      0.60      0.48        50
weighted avg       0.78      0.52      0.45        50

